# Телеграм бот

In [4]:
# !pip uninstall -y tensorflow tensorflow-gpu
# !pip install numpy scipy librosa unidecode inflect librosa transformers
# !pip install deeppavlov

In [5]:
# !pip install python-telegram-bot --upgrade

In [6]:
# pip install dialogflow

In [7]:
from telegram.ext  import Updater, CommandHandler, MessageHandler, Filters
import dialogflow

In [8]:
my_token = ''

In [9]:
updater = Updater(token=my_token) # Токен API к Telegram
dispatcher = updater.dispatcher

In [10]:
# Обработка команд
def startCommand(bot, update):
    bot.send_message(chat_id=update.message.chat_id, text='Добрый день')
    
def textMessage(bot, update):
    response = 'Ваше сообщение принял ' + update.message.text # формируем текст ответа
    bot.send_message(chat_id=update.message.chat_id, text=response)

In [11]:
import logging

from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

import dialogflow

In [12]:
import logging
# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)

logger = logging.getLogger()

# RuBERT for question answering.
Here we will use the model pre-trained on the SberQuAD dataset from the [SDSJ-2017 challenge problem B](https://github.com/sberbank-ai/data-science-journey-2017/tree/master/problem_B).

In [13]:
# !python -m deeppavlov install squad_ru_rubert

In [14]:
from deeppavlov import build_model, configs
model_ru = build_model(configs.squad.squad_ru_rubert, download=True)

/usr/local/lib/python3.7/dist-packages/cryptography/hazmat/backends/openssl/x509.py:18: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  utils.DeprecatedIn35,
2021-11-01 18:39:45.295 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/deeppavlov_data/squad_model_ru_rubert.tar.gz download because of matching hashes
2021-11-01 18:39:45,295 - deeppavlov.download - INFO - Skipped http://files.deeppavlov.ai/deeppavlov_data/squad_model_ru_rubert.tar.gz download because of matching hashes
2021-11-01 18:39:50.94 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v1.tar.gz download because of matching hashes
2021-11-01 18:39:50,094 - deeppavlov.download - INFO - Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v1.tar.gz download because of matching hash

The following text is copied from [habr post on Crew Dragon flight](https://habr.com/ru/news/t/504642/).

In [15]:
text = """Первая многоразовая ступень ракеты-носителя Falcon 9 успешно отделилась через две с половиной минуты после старта и автоматически приземлилась на плавучую платформу Of Course I Still Love You у берегов Флориды. Через 12 минут после запуска космический корабль Crew Dragon вышел на расчетную орбиту и отделился от второй ступени ракеты.

Сближение корабля Crew Dragon с Международной космической станцией запланировано на 31 мая. К стыковочному адаптеру на узловом модуле «Гармония» американского сегмента МКС Crew Dragon должен причалить в ручном или, при необходимости, в автоматическом режиме. Эта процедура запланирована на 10:29 по времени Восточного побережья США (17:29 по московскому времени).

В испытательном полете DM2 астронавт Херли является командиром космического корабля (spacecraft commander), а его напарник Бенкен — командир по операциям стыковки и расстыковки (joint operations commander). Фактически это означает, что именно Херли управляет Crew Dragon в полете к МКС, к которой они должны пристыковаться в течение суток после старта. Херли и Бенкен также будут выполнять необходимые для сертификации НАСА проверки систем корабля в полете.

Во время полета Херли и Бенкен провели небольшую экскурсию по Crew Dragon."""

# Source: https://habr.com/ru/news/t/504642/

In [16]:
print(text)

Первая многоразовая ступень ракеты-носителя Falcon 9 успешно отделилась через две с половиной минуты после старта и автоматически приземлилась на плавучую платформу Of Course I Still Love You у берегов Флориды. Через 12 минут после запуска космический корабль Crew Dragon вышел на расчетную орбиту и отделился от второй ступени ракеты.

Сближение корабля Crew Dragon с Международной космической станцией запланировано на 31 мая. К стыковочному адаптеру на узловом модуле «Гармония» американского сегмента МКС Crew Dragon должен причалить в ручном или, при необходимости, в автоматическом режиме. Эта процедура запланирована на 10:29 по времени Восточного побережья США (17:29 по московскому времени).

В испытательном полете DM2 астронавт Херли является командиром космического корабля (spacecraft commander), а его напарник Бенкен — командир по операциям стыковки и расстыковки (joint operations commander). Фактически это означает, что именно Херли управляет Crew Dragon в полете к МКС, к которой о

In [17]:
model_ru([text], ['Когда отделилась первая ступень?'])

[['через две с половиной минуты после старта'], [72], [2055731.625]]

In [18]:
model_ru([text], ['На какую дату запланирована стыковка?'])

[['на 31 мая'], [418], [31752.884765625]]

In [19]:
model_ru([text], ['Кто участвует в полете?'])

[['астронавт Херли'], [729], [139.62789916992188]]

In [20]:
model_ru([text], ['Кто участвует в полете кроме астронавта Херли?'])

[['Бенкен'], [1063], [13.483261108398438]]

In [28]:
reply_text = model_ru([text], ['Кто участвует в полете кроме астронавта Херли?'])
reply_text[0][0].encode('utf8')

b'\xd0\x91\xd0\xb5\xd0\xbd\xd0\xba\xd0\xb5\xd0\xbd'

# Telegram + RuBERT for question answering.

In [29]:
# Define a few command handlers. These usually take the two arguments update and
# context. Error handlers also receive the raised TelegramError object in error.
def start(update: Update, context: CallbackContext):
    update.message.reply_text('Hi!')

def echo(update: Update, context: CallbackContext):
    txt = update.message.text

    update.message.reply_text('Ваше сообщение! ' + update.message.text)
    reply_text = model_ru([text], [update.message.text])

    #.encode('utf8')
    reply_text = reply_text[0][0] 
    
    update.message.reply_text(reply_text)

In [ ]:
updater = Updater(my_token, use_context=True)
dispatcher = updater.dispatcher

# on different commands - answer in Telegram
dispatcher.add_handler(CommandHandler("start", start))
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))

# Start the Bot
updater.start_polling()
updater.idle()

2021-11-01 18:43:47,555 - apscheduler.scheduler - INFO - Scheduler started
2021-11-01 18:44:20,329 - telegram.ext.dispatcher - ERROR - No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/telegram/ext/dispatcher.py", line 555, in process_update
    handler.handle_update(update, self, check, context)
  File "/usr/local/lib/python3.7/dist-packages/telegram/ext/handler.py", line 198, in handle_update
    return self.callback(update, context)
  File "<ipython-input-29-88d5aff9684f>", line 15, in echo
    update.message.reply_text(reply_text)
  File "/usr/local/lib/python3.7/dist-packages/telegram/message.py", line 775, in reply_text
    entities=entities,
  File "/usr/local/lib/python3.7/dist-packages/telegram/bot.py", line 130, in decorator
    result = func(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/telegram/bot.py", line 521, in send_message
    api_kwargs=api_kwargs,
  File "/usr/loca